In [13]:
import pandas as pd

# Load the dataset
data = pd.read_csv("./Datasets/test.csv")

# Display the first few rows of the dataset
data.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,yes,1984,1.358411,103.891722,0.0,lorong ah soo,hougang,north-east region
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,yes,1999,1.446343,103.820817,0.0,sembawang central,sembawang,north region
2,2023-07,clementi,708,Clementi West Street 2,4-room,new generation,91.0,yes,1980,1.305719,103.762168,0.0,clementi west,clementi,west region
3,2021-08,jurong east,351,Jurong East Street 31,3 room,model a,74.0,yes,1986,1.344832,103.730778,0.0,yuhua west,jurong east,west region
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,yes,1983,1.345437,103.735241,0.0,yuhua west,jurong east,west region


In [14]:
# Convert textual columns to lowercase for normalization
textual_columns = ['town', 'block', 'street_name', 'flat_type', 'flat_model', 'furnished', 'subzone', 'planning_area', 'region']

data[textual_columns] = data[textual_columns].apply(lambda x: x.str.lower())

# Display the first few rows to verify changes
data.head()

,rent_approval_date,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region
0,2023-01,hougang,245,hougang street 22,5-room,improved,121.0,yes,1984,1.358411,103.891722,0.0,lorong ah soo,hougang,north-east region
1,2022-09,sembawang,316,sembawang vista,4-room,model a,100.0,yes,1999,1.446343,103.820817,0.0,sembawang central,sembawang,north region
2,2023-07,clementi,708,clementi west street 2,4-room,new generation,91.0,yes,1980,1.305719,103.762168,0.0,clementi west,clementi,west region
3,2021-08,jurong east,351,jurong east street 31,3 room,model a,74.0,yes,1986,1.344832,103.730778,0.0,yuhua west,jurong east,west region
4,2022-03,jurong east,305,jurong east street 32,5-room,improved,121.0,yes,1983,1.345437,103.735241,0.0,yuhua west,jurong east,west region


In [15]:
# Extract year and month from the 'rent_approval_date' column
data['rent_approval_year'] = pd.to_datetime(data['rent_approval_date']).dt.year
data['rent_approval_month'] = pd.to_datetime(data['rent_approval_date']).dt.month

# Drop the original 'rent_approval_date' column
data = data.drop(columns=['rent_approval_date'])

# Display the modified dataset
data.head()

,town,block,street_name,flat_type,flat_model,floor_area_sqm,furnished,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,rent_approval_year,rent_approval_month
0,hougang,245,hougang street 22,5-room,improved,121.0,yes,1984,1.358411,103.891722,0.0,lorong ah soo,hougang,north-east region,2023,1
1,sembawang,316,sembawang vista,4-room,model a,100.0,yes,1999,1.446343,103.820817,0.0,sembawang central,sembawang,north region,2022,9
2,clementi,708,clementi west street 2,4-room,new generation,91.0,yes,1980,1.305719,103.762168,0.0,clementi west,clementi,west region,2023,7
3,jurong east,351,jurong east street 31,3 room,model a,74.0,yes,1986,1.344832,103.730778,0.0,yuhua west,jurong east,west region,2021,8
4,jurong east,305,jurong east street 32,5-room,improved,121.0,yes,1983,1.345437,103.735241,0.0,yuhua west,jurong east,west region,2022,3


In [16]:
# Standardize the flat_type values by replacing " room" with "-room"
data['flat_type'] = data['flat_type'].str.replace(" room", "-room")

# Check the updated unique values in the 'flat_type' column
updated_flat_types = data['flat_type'].unique()

updated_flat_types

array(['5-room', '4-room', '3-room', 'executive', '2-room'], dtype=object)

In [17]:
# Identify columns with only one unique value
columns_to_drop = [col for col in data.columns if data[col].nunique() == 1]

# Drop these columns from the dataset
data = data.drop(columns=columns_to_drop)

# Display the updated dataset columns
remaining_columns = data.columns
remaining_columns

Index(['town', 'block', 'street_name', 'flat_type', 'flat_model',
       'floor_area_sqm', 'lease_commence_date', 'latitude', 'longitude',
       'subzone', 'planning_area', 'region', 'rent_approval_year',
       'rent_approval_month'],
      dtype='object')

In [18]:
# Drop the 'block', 'street names' columns as it is too granular and may not generalize well
data = data.drop(columns=['block', 'street_name'])

In [19]:
# Load the first dataset
sg_coe_prices = pd.read_csv("./Datasets/auxiliary-data/sg-coe-prices.csv")
sg_coe_prices.head()

,year,category,month,bidding,price,quota,bids
0,2023,a,july,2,95202,581,728
1,2023,a,july,1,97000,588,756
2,2023,a,june,2,96206,586,751
3,2023,a,june,1,98001,581,881
4,2023,a,may,2,92000,596,936


In [20]:
# Convert month names in sg_coe_prices to numerical values for merging
month_map = {
    'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6,
    'july': 7, 'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12
}
sg_coe_prices['month_num'] = sg_coe_prices['month'].map(month_map)
sg_coe_prices.rename(columns={'price': 'coe_price'}, inplace=True)

# Aggregate COE prices by taking the average for each month and year across all categories
average_coe_prices = sg_coe_prices.groupby(['year', 'month_num'])['coe_price'].mean().reset_index()

# Merge the aggregated COE prices back into the rental data
data = pd.merge(data, average_coe_prices,
                           left_on=['rent_approval_year', 'rent_approval_month'],
                           right_on=['year', 'month_num'],
                           how='left').drop(columns=['year', 'month_num'])

data.head()

,town,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,rent_approval_year,rent_approval_month,coe_price
0,hougang,5-room,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,2023,1,92845.000
1,sembawang,4-room,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,2022,9,92942.625
2,clementi,4-room,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,2023,7,104442.375
3,jurong east,3-room,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,2021,8,51997.375
4,jurong east,5-room,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,2022,3,79016.000


In [21]:
# Loading the datasets
sg_stock_prices = pd.read_csv('./Datasets/auxiliary-data/INDEX_SG_XSES_STI.csv')

# Displaying the first few rows of each dataset
sg_stock_prices.head()

,Date,Open,High,Low,Close
0,10/20/2023,"3,076.69","3,100.44","3,074.07","3,076.69"
1,10/19/2023,"3,111.78","3,099.60","3,099.60","3,099.60"
2,10/18/2023,"3,167.63","3,136.62","3,136.62","3,136.62"
3,10/17/2023,"3,185.79","3,171.83","3,171.83","3,171.83"
4,10/16/2023,"3,163.89","3,179.39","3,157.63","3,163.89"


In [22]:
# Convert the Date column in the index_data to datetime format
sg_stock_prices['Date'] = pd.to_datetime(sg_stock_prices['Date'])

# Extract year and month from the Date column in index_data for merging
sg_stock_prices['Year'] = sg_stock_prices['Date'].dt.year
sg_stock_prices['Month'] = sg_stock_prices['Date'].dt.month

# Aggregate the stock index data using mean, min, and max for each month and year
for col in ['Open', 'Low', 'Close', 'High']:
    sg_stock_prices[col] = sg_stock_prices[col].str.replace(',','').astype(float)

# Re-run the aggregation
mean_aggregated = sg_stock_prices.groupby(['Year', 'Month']).mean().reset_index()
min_aggregated = sg_stock_prices.groupby(['Year', 'Month']).min().reset_index()
max_aggregated = sg_stock_prices.groupby(['Year', 'Month']).max().reset_index()

In [23]:
# From the above correlations, the Close value with the Min aggregation method has the highest correlation with monthly_rent. Therefore, if we were to choose a single aggregated stock index value as a feature, the Close price with the Min aggregation would be the most suitable based on its correlation with the target variable.
# Merge the train_data with min_aggregated data using only the 'Close' column
data = pd.merge(data, min_aggregated[['Year', 'Month', 'Close']],
                            left_on=['rent_approval_year', 'rent_approval_month'],
                            right_on=['Year', 'Month'], how='left')

# Rename the 'Close' column for clarity
data.rename(columns={'Close': 'Stock_Price'}, inplace=True)

data.head()

,town,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,rent_approval_year,rent_approval_month,coe_price,Year,Month,Stock_Price
0,hougang,5-room,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,2023,1,92845.000,2023,1,3242.46
1,sembawang,4-room,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,2022,9,92942.625,2022,9,3115.08
2,clementi,4-room,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,2023,7,104442.375,2023,7,3139.47
3,jurong east,3-room,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,2021,8,51997.375,2021,8,3055.05
4,jurong east,5-room,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,2022,3,79016.000,2022,3,3148.86


In [24]:
import numpy as np

# 直线距离
def haversine_distances(df_lats, df_longs, dst_lats, dst_longs):
    R = 6371000  # Earth's average radius in meters

    lat = np.radians(dst_lats) - np.radians(df_lats[:, np.newaxis])
    lon = np.radians(dst_longs) - np.radians(df_longs[:, np.newaxis])

    a = np.sin(lat / 2.0) ** 2 + np.cos(np.radians(df_lats[:, np.newaxis])) * np.cos(np.radians(dst_lats)) * np.sin(lon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

def add_distance_features(train_df, poi_df, prefix):
    distances = haversine_distances(train_df['latitude'].values, train_df['longitude'].values, poi_df['latitude'].values, poi_df['longitude'].values)

    train_df[prefix + '_nearest_distance'] = np.min(distances, axis=1).round()
    nearest_index = np.argmin(distances, axis=1)
    train_df[prefix + '_nearest'] = poi_df['name'].iloc[nearest_index].values
    train_df[prefix + '_count_within_1km'] = np.sum(distances <= 1000, axis=1)

# Load data
mrt_file = pd.read_csv("./Datasets/auxiliary-data/sg-mrt-existing-stations.csv")
shoppingmall_file = pd.read_csv("./Datasets/auxiliary-data/sg-shopping-malls.csv")

# Add distance features for MRT stations and shopping malls
add_distance_features(data, mrt_file, 'mrt')
add_distance_features(data, shoppingmall_file, 'mall')

data.head(10), data.shape

(            town  flat_type         flat_model  floor_area_sqm  \
 0        hougang     5-room           improved           121.0   
 1      sembawang     4-room            model a           100.0   
 2       clementi     4-room     new generation            91.0   
 3    jurong east     3-room            model a            74.0   
 4    jurong east     5-room           improved           121.0   
 5       clementi     3-room     new generation            67.0   
 6        punggol     5-room  premium apartment           110.0   
 7       tampines     5-room               dbss           108.0   
 8    jurong west  executive  premium apartment           133.0   
 9  marine parade     5-room           standard           120.0   
 
    lease_commence_date  latitude   longitude              subzone  \
 0                 1984  1.358411  103.891722        lorong ah soo   
 1                 1999  1.446343  103.820817    sembawang central   
 2                 1980  1.305719  103.762168      

In [25]:
# Load datasets
planned_mrt_file = pd.read_csv("./Datasets/auxiliary-data/sg-mrt-planned-stations.csv")

# Filter planned MRT stations that will open within the next year
upcoming_mrts = planned_mrt_file[planned_mrt_file['opening_year'] == "2024"]

# Combine the existing MRT data with the upcoming MRT data
combined_mrt_file = pd.concat([mrt_file, upcoming_mrts], ignore_index=True)

add_distance_features(data, combined_mrt_file, 'mrt_planned')
data.head(20)

,town,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,...,Stock_Price,mrt_nearest_distance,mrt_nearest,mrt_count_within_1km,mall_nearest_distance,mall_nearest,mall_count_within_1km,mrt_planned_nearest_distance,mrt_planned_nearest,mrt_planned_count_within_1km
0,hougang,5-room,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,...,3242.46,820.0,Kovan,1,739.0,Heartland Mall,1,820.0,Kovan,1
1,sembawang,4-room,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,...,3115.08,308.0,Sembawang,1,255.0,Sun Plaza,2,308.0,Sembawang,1
2,clementi,4-room,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,...,3139.47,1098.0,Commonwealth,0,498.0,West Coast Plaza,1,1098.0,Commonwealth,0
3,jurong east,3-room,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,...,3055.05,363.0,Chinese Garden,1,1596.0,Taman Jurong Shopping Centre,0,363.0,Chinese Garden,1
4,jurong east,5-room,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,...,3148.86,449.0,Chinese Garden,1,1461.0,JCube,0,449.0,Chinese Garden,1
5,clementi,3-room,new generation,67.0,1980,1.307637,103.761326,clementi west,clementi,west region,...,3134.25,936.0,Commonwealth,2,696.0,West Coast Plaza,3,936.0,Commonwealth,2
6,punggol,5-room,premium apartment,110.0,2005,1.397319,103.916019,waterway east,punggol,north-east region,...,3055.02,1741.0,Punggol,0,476.0,Punggol Plaza,2,1741.0,Punggol,0
7,tampines,5-room,dbss,108.0,2008,1.356968,103.938668,tampines east,tampines,east region,...,3221.67,523.0,Tampines,2,419.0,Our Tampines Hub,4,523.0,Tampines,2
8,jurong west,executive,premium apartment,133.0,2001,1.338605,103.700079,jurong west central,jurong west,west region,...,3072.97,316.0,Pioneer,2,464.0,Pioneer Mall,2,316.0,Pioneer,2
9,marine parade,5-room,standard,120.0,1975,1.303476,103.915412,marine parade,marine parade,central region,...,3139.47,1973.0,Kembangan,0,1061.0,Parkway Parade,0,362.0,Marine Terrace,1


In [26]:
# Get the number of rows for each dataset
num_rows_train = data.shape[0]
num_rows_index = data.shape[0]

num_rows_train, num_rows_index

(30000, 30000)

In [27]:
updated_ordinal_mapping = {
    '2-room': 1,
    '3-room': 2,
    '4-room': 3,
    '5-room': 4,
    'executive': 5
}

# Re-encode the flat_type column using the updated ordinal mapping
data['flat_type_ordinal'] = data['flat_type'].map(updated_ordinal_mapping)

# Display the original and updated ordinal encoded 'flat_type' columns for comparison
data[['flat_type', 'flat_type_ordinal']].head()

,flat_type,flat_type_ordinal
0,5-room,4
1,4-room,3
2,4-room,3
3,3-room,2
4,5-room,4


In [28]:
# Drop the 'flat_type' column
data = data.drop(columns=['flat_type'])

# Display the first few rows of the dataset to confirm
data.head()

,town,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,rent_approval_year,...,mrt_nearest_distance,mrt_nearest,mrt_count_within_1km,mall_nearest_distance,mall_nearest,mall_count_within_1km,mrt_planned_nearest_distance,mrt_planned_nearest,mrt_planned_count_within_1km,flat_type_ordinal
0,hougang,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,2023,...,820.0,Kovan,1,739.0,Heartland Mall,1,820.0,Kovan,1,4
1,sembawang,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,2022,...,308.0,Sembawang,1,255.0,Sun Plaza,2,308.0,Sembawang,1,3
2,clementi,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,2023,...,1098.0,Commonwealth,0,498.0,West Coast Plaza,1,1098.0,Commonwealth,0,3
3,jurong east,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,2021,...,363.0,Chinese Garden,1,1596.0,Taman Jurong Shopping Centre,0,363.0,Chinese Garden,1,2
4,jurong east,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,2022,...,449.0,Chinese Garden,1,1461.0,JCube,0,449.0,Chinese Garden,1,4


In [29]:
# Compute centroid of all properties(marina bay centroid)
marina_centroid_lat = 1.287515
marina_centroid_lon = 103.866641

# Convert the latitudes and longitudes to numpy arrays
df_lats = data['latitude'].values
df_longs = data['longitude'].values
dst_lats = np.array([marina_centroid_lat])
dst_longs = np.array([marina_centroid_lon])

# Calculate distance of each property to marina bay
distances = haversine_distances(df_lats, df_longs, dst_lats, dst_longs)
data['distance_to_centroid_marina_bay'] = distances[:, 0] / 1000

# Lease Duration
current_year = 2023
data['lease_duration'] = current_year - data['lease_commence_date']

# # 2. Categorical Features
# 
# # 2.1 High Cardinality
# # Drop the 'subzone' column
# data = data.drop(columns=['subzone'])
# 
# # 2.2 Drop Redundant Columns
# # Drop 'mrt_nearest' and other high cardinality columns
# data = data.drop(columns=['mrt_nearest', 'mall_nearest', 'mrt_plus_nearest'])

In [30]:
train_with_schoolscore = pd.read_csv("./Datasets/auxiliary-data/test_with_schoolscore.csv")

# Merge based on latitude and longitude
are_latitudes_equal = (data['latitude'].values == train_with_schoolscore['latitude'].values).all()
are_longitudes_equal = (data['longitude'].values == train_with_schoolscore['longitude'].values).all()

if are_latitudes_equal == are_longitudes_equal == False:
    raise Exception('The latitude and longitude values are not equal between the two datasets')
data['school_score'] = train_with_schoolscore['school_score']

# Display the first few rows of the updated train dataset
data.head()

,town,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,subzone,planning_area,region,rent_approval_year,...,mall_nearest_distance,mall_nearest,mall_count_within_1km,mrt_planned_nearest_distance,mrt_planned_nearest,mrt_planned_count_within_1km,flat_type_ordinal,distance_to_centroid_marina_bay,lease_duration,school_score
0,hougang,improved,121.0,1984,1.358411,103.891722,lorong ah soo,hougang,north-east region,2023,...,739.0,Heartland Mall,1,820.0,Kovan,1,4,8.361822,39,376.7
1,sembawang,model a,100.0,1999,1.446343,103.820817,sembawang central,sembawang,north region,2022,...,255.0,Sun Plaza,2,308.0,Sembawang,1,3,18.380806,24,312.0
2,clementi,new generation,91.0,1980,1.305719,103.762168,clementi west,clementi,west region,2023,...,498.0,West Coast Plaza,1,1098.0,Commonwealth,0,3,11.788992,43,255.4
3,jurong east,model a,74.0,1986,1.344832,103.730778,yuhua west,jurong east,west region,2021,...,1596.0,Taman Jurong Shopping Centre,0,363.0,Chinese Garden,1,2,16.392948,37,284.8
4,jurong east,improved,121.0,1983,1.345437,103.735241,yuhua west,jurong east,west region,2022,...,1461.0,JCube,0,449.0,Chinese Garden,1,4,15.964070,40,287.8


In [31]:
# Save the cleaned and analyzed dataset to a new CSV file
output_filepath = "./Datasets/cleaned_test.csv"
data.to_csv(output_filepath, index=False)